In [92]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import lstsq as the_root_is
from numpy import exp as ℯ

In [93]:
# Data preparation
rf = 0.06
K = 1.1
S3 = np.array([1.34, 1.54, 1.03, 0.92, 1.52, 0.9, 1.01, 1.34])
S2 = np.array([1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22])
S1 = np.array([1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88])
S0 = np.ones_like(S1)

In [94]:
# Create the Cash-flow
def Exercise(Array):
    CF = np.zeros_like(Array)
    for i in range(len(Array)):
        if Array[i] > K:
            CF[i] = 0
        else:
            CF[i] = K - Array[i]
    return CF

In [95]:
# The simple regression formula
def Regression(Y, x):
    x_modified = []
    Y_modified = []
    for i in range(len(x)):
        if x[i] < K:
            x_modified.append(x[i])
            Y_modified.append(Exercise(Y)[i]*ℯ(-1*rf))

    X = np.column_stack([np.array(x_modified)**i for i in range(3)])
    return the_root_is(X, Y_modified, rcond=None)[0]

In [96]:
# Calculate the THEO of Stock Value
def THEO(Y, x):
    x_modified = []
    for i in range(len(x)):
        if x[i] < K:
            x_modified.append(x[i])
    X = np.column_stack([np.array(x_modified)**i for i in range(3)])
    return X@Regression(Y,x)

In [124]:
# Define whether to exercise
def Cashflow(S_after, S_before):
    Decision = []
    count = 0
    for i in range(len(S_after)):
        if S_before[i] > K:
            Decision.append(0)
            count += 1
        elif THEO(S_after, S_before)[i-count] > Exercise(S_before)[i]:
            Decision.append(0)
        else: Decision.append(round(Exercise(S_before)[i], 2))
    return Decision

In [147]:
def Option_Cashflow_Matrix_Generator(s1, s2, s3):
    CF1 = Cashflow(S2, S1)
    CF2 = Cashflow(S3, S2)
    CF3 = Exercise(S3)
    for i in range(len(CF1)):
        if CF2[i] != 0:
            CF3[i] = 0
        if CF1[i] != 0:
            CF2[i] = 0
    Option_Cashflow_Matrix = np.array([range(1, len(CF1)+1), CF1, CF2, CF3])
    return Option_Cashflow_Matrix.T

In [148]:
Option_Cashflow_Matrix_Generator(S1, S2,  S3)

array([[1.  , 0.  , 0.  , 0.  ],
       [2.  , 0.  , 0.  , 0.  ],
       [3.  , 0.  , 0.  , 0.07],
       [4.  , 0.17, 0.  , 0.  ],
       [5.  , 0.  , 0.  , 0.  ],
       [6.  , 0.34, 0.  , 0.  ],
       [7.  , 0.18, 0.  , 0.  ],
       [8.  , 0.22, 0.  , 0.  ]])